### Temp Notebook

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
df_twogram_pair = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/1-Twogram With Different Word List Pair/Turkish/Turkish_200_Word_Twogram_With_28_And_172_Word_Pair_All.xlsx")
df_twogram_pair

In [ ]:
df_file = pd.read_excel("twogram_selected.xlsx")
df_file

In [ ]:
df_file["twogram"].nunique()

In [ ]:
df_cross_check = pd.merge(df_twogram_pair, df_file, how="inner", on="twogram")
df_cross_check.drop_duplicates(inplace=True)
df_cross_check.reset_index(drop=True, inplace=True)
df_cross_check

In [ ]:
df_cross_check.to_excel("Basics Cover the Most Master File 28-172 Word Twogram.xlsx", index=False)

In [ ]:
df_not_in_cross = df_file[~df_file["twogram"].isin(df_cross_check["twogram"].to_list())]
df_not_in_cross.drop_duplicates(inplace=True)
df_not_in_cross.reset_index(drop=True, inplace=True)
df_not_in_cross

In [ ]:
df_not_in_cross.to_excel("Basics Cover the Most Master File 172 Word Twogram.xlsx", index=False)

#### Selected Twogram In Threegram

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 200  # 28 native word end index

In [13]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"search_string",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [6]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [7]:
word_list = df_word_select["word"].to_list()

In [8]:
df_selected_twogram = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/\
Deploy2 Result Manuel/Turkish/Turkish_200_Word_Twogram_With_28_And_172_Word_Pair_Selected_Manuel.xlsx")
df_selected_twogram

,unique_twogram
0,biz de
1,yok artık
2,söyle bana
3,bir şeyler
4,bu doğru
...,...
68,daha fazla
69,seni küçük
70,oh evet
71,ne zaman


In [9]:
df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Result/N Gram/Merge/Threegram_Merge.csv")  
df_threegram = df_threegram.loc[:,["threegram","frequency"]]
df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [10]:
d_list3  = df_threegram.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,başka bir şey,75193
4,hayır hayır hayır,69443
...,...,...
109758,bu değil şimdi,10
109759,gece içinde üç,10
109760,bu değil gibi,10
109761,fazla adam olacak,10


In [14]:
df_threegram_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109763 entries, 0 to 109762
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   threegram  109763 non-null  object
 1   frequency  109763 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ MB


In [15]:
df_selected_twogram_in_threegram = word_in_wordgroup(df_selected_twogram["unique_twogram"].to_list(), df_threegram_select, "threegram", 6, simple=False)
df_selected_twogram_in_threegram

,search_string,threegram,frequency
0,biz de,ve biz de,3286
1,biz de,biz de öyle,3059
2,biz de,biz de bunu,1945
3,biz de,biz de onu,1660
4,biz de,biz de bu,1645
...,...,...,...
433,hayır efendim,hayır efendim ben,195
434,hayır efendim,hayır efendim ama,183
435,hayır efendim,hayır efendim bu,177
436,hayır efendim,hayır efendim sadece,123


In [16]:
df_selected_twogram_in_threegram.to_excel("Turkish_200_Word_Twogram_With_28_And_172_Word_Pair_In_Threegram.xlsx", index=False)